## 1. Setup and Imports

In [1]:
# pip install duckdb

In [2]:
# pip install pandas

In [ ]:
import sys
import os

# Add parent directory to path to import project modules
sys.path.insert(0, os.path.abspath('..'))

import duckdb
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import warnings
warnings.filterwarnings('ignore')

# Import project modules
from account import Account
from config_loader import CONFIG

print("✓ Imports successful")
print(f"Current time: {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')}")

✓ Imports successful
Current time: 2025-11-23 13:46:17


## 2. Configuration

In [4]:
# Select trading mode: 'paper' or 'live'
TRADE_MODE = 'live'  # Change to 'live' for live trading

# Database selection
DB_FILE = 'live_transaction.db' if TRADE_MODE == 'live' else 'paper_transaction.db'
DB_PATH = os.path.abspath(os.path.join('..', DB_FILE))

# Check if database exists
if not os.path.exists(DB_PATH):
    print(f"⚠️  Warning: Database not found at {DB_PATH}")
    print(f"Looking for database files in parent directory...")
    parent_dir = os.path.abspath('..')
    db_files = [f for f in os.listdir(parent_dir) if f.endswith('.db')]
    if db_files:
        print(f"Found database files: {db_files}")
    else:
        print("No database files found. Please create transactions first.")
else:
    print(f"✓ Database found: {DB_PATH}")

print(f"\nMode: {TRADE_MODE.upper()}")
print(f"Database: {DB_PATH}")
print(f"API Base URL: {CONFIG.get('BASE_URL', 'https://api.woox.io')}")

✓ Database found: /Users/honcy/myproj/woox/live_transaction.db

Mode: LIVE
Database: /Users/honcy/myproj/woox/live_transaction.db
API Base URL: https://api.woox.io


## 3. Account Overview

In [5]:
# Change to parent directory for Account to find database
original_dir = os.getcwd()
os.chdir('..')

try:
    # Initialize account
    account = Account(trade_mode=TRADE_MODE)

    print("=" * 60)
    print(f"ACCOUNT SUMMARY - {TRADE_MODE.upper()} MODE")
    print("=" * 60)
    
    # Get API balance (for live mode with API credentials)
    api_balance = account.get_api_balance()
    if api_balance:
        print("\n📊 LIVE API BALANCE:")
        print("-" * 60)
        
        # Handle different response formats
        balances = None
        if 'balances' in api_balance:
            balances = api_balance['balances']
        elif 'holding' in api_balance:
            # Single balance object
            balances = [api_balance]
        elif isinstance(api_balance, list):
            # Direct list of balances
            balances = api_balance
        
        if balances:
            df_balance = pd.DataFrame(balances)
            
            # Ensure required columns exist
            if 'holding' in df_balance.columns:
                df_balance['holding'] = pd.to_numeric(df_balance['holding'], errors='coerce')
                
                # Handle frozen column - ensure it's a Series
                if 'frozen' in df_balance.columns:
                    df_balance['frozen'] = pd.to_numeric(df_balance['frozen'], errors='coerce').fillna(0)
                else:
                    df_balance['frozen'] = 0.0
                
                df_balance['available'] = df_balance['holding'] - df_balance['frozen']
                
                # Filter non-zero balances
                df_balance = df_balance[df_balance['holding'] > 0].copy()
                
                if len(df_balance) > 0:
                    # Display formatted balance
                    print(f"\n{'Token':<10} {'Holding':<18} {'Frozen':<18} {'Available':<18}")
                    print("-" * 64)
                    for _, row in df_balance.iterrows():
                        token = row.get('token', row.get('coin', 'UNKNOWN'))
                        holding = row['holding']
                        frozen = row['frozen']
                        available = row['available']
                        print(f"{token:<10} {holding:<18.8f} {frozen:<18.8f} {available:<18.8f}")
                    
                    # Summary - fix the comparison
                    token_col = 'token' if 'token' in df_balance.columns else 'coin'
                    usdt_balance = df_balance[df_balance[token_col] == 'USDT']
                    if len(usdt_balance) > 0:
                        total_usdt = usdt_balance['holding'].sum()
                        print(f"\n💰 Total USDT Balance: ${total_usdt:,.2f}")
                    
                    print(f"\n📊 Total Assets: {len(df_balance)} tokens with non-zero balance")
                else:
                    print("\nNo balances with holdings > 0")
            else:
                print("\n⚠️  API response missing 'holding' field")
                print(f"Response keys: {list(df_balance.columns)}")
                print(f"Sample data:\n{df_balance.head()}")
        else:
            print("\n⚠️  No balance data in API response")
            print(f"Response structure: {list(api_balance.keys())}")
    else:
        if TRADE_MODE == 'live':
            print("\n⚠️  API balance not available")
            print("Please check:")
            print("  1. API credentials are set (WOOX_API_KEY, WOOX_API_SECRET)")
            print("  2. API keys have balance read permissions")
            print("  3. Network connectivity to WOOX API")
        else:
            print("\n📝 Paper trading mode - API balance not applicable")
    
    # Get transaction summary from database
    print("\n" + "=" * 60)
    print("TRANSACTION SUMMARY FROM DATABASE")
    print("=" * 60)
    tx_summary = account.get_transaction_summary()
    print(f"Total Transactions: {tx_summary.get('total_transactions', 0)}")
    print(f"Total Buy Orders: {tx_summary.get('total_buy', 0)}")
    print(f"Total Sell Orders: {tx_summary.get('total_sell', 0)}")
    print(f"Realized P&L: ${tx_summary.get('realized_pnl', 0):.2f}")
    
    # Get open positions
    positions = account.get_open_positions()
    if positions:
        print("\n" + "=" * 60)
        print("OPEN POSITIONS")
        print("=" * 60)
        df_positions = pd.DataFrame(positions)
        print(df_positions.to_string(index=False))
    else:
        print("\n📭 No open positions")
        
except Exception as e:
    print(f"❌ Error retrieving account information: {e}")
    import traceback
    traceback.print_exc()
finally:
    # Change back to portfolio_analysis directory
    os.chdir(original_dir)

2025-11-23 13:46:17,686 - Account - INFO - Account initialized in LIVE mode


ACCOUNT SUMMARY - LIVE MODE

📊 LIVE API BALANCE:
------------------------------------------------------------

No balances with holdings > 0

TRANSACTION SUMMARY FROM DATABASE
Total Transactions: 0
Total Buy Orders: 0
Total Sell Orders: 0
Realized P&L: $0.00

OPEN POSITIONS
     symbol  quantity  avg_entry_price  trade_count
TESTTICKER2      10.0            333.0            1


## 4. Transaction History Analysis

In [6]:
# Connect to database and load transactions
if not os.path.exists(DB_PATH):
    print(f"❌ Error: Database file not found at {DB_PATH}")
    print("Please create some transactions first before running this analysis.")
    df_transactions = pd.DataFrame()
else:
    conn = duckdb.connect(DB_PATH, read_only=True)

    # Get all transactions from trades table
    query = """
    SELECT 
        trade_datetime as timestamp,
        symbol,
        trade_type as side,
        order_type,
        price,
        quantity,
        code as status,
        acct_id,
        proceeds,
        commission,
        fee
    FROM trades
    ORDER BY trade_datetime DESC
    """

    df_transactions = conn.execute(query).fetchdf()

    print(f"Total transactions: {len(df_transactions)}")
    if len(df_transactions) > 0:
        print(f"\nRecent transactions (last 10):")
        display_cols = ['timestamp', 'symbol', 'side', 'price', 'quantity', 'status']
        print(df_transactions[display_cols].head(10).to_string(index=False))
    else:
        print("\nNo transactions found in database.")

Total transactions: 3

Recent transactions (last 10):
          timestamp      symbol side  price  quantity status
2025-11-13 11:05:36 TESTTICKER2  BUY 333.00      10.0      O
2025-11-13 10:23:52 TESTTICKER1 SELL 337.41     -10.0      C
2025-11-13 09:31:05 TESTTICKER1  BUY 335.00      10.0      O


## 5. Transaction Statistics

In [7]:
# Filter completed transactions
df_completed = df_transactions[df_transactions['status'].isin(['FILLED', 'COMPLETED'])].copy()

if len(df_completed) > 0:
    print("=" * 60)
    print("TRANSACTION STATISTICS")
    print("=" * 60)
    
    # By symbol
    print("\nTransactions by Symbol:")
    symbol_stats = df_completed.groupby('symbol').agg({
        'order_id': 'count',
        'quantity': 'sum',
        'price': ['mean', 'min', 'max']
    }).round(2)
    symbol_stats.columns = ['Count', 'Total Qty', 'Avg Price', 'Min Price', 'Max Price']
    print(symbol_stats)
    
    # By side
    print("\nTransactions by Side:")
    side_stats = df_completed.groupby('side').agg({
        'order_id': 'count',
        'quantity': 'sum'
    }).round(2)
    side_stats.columns = ['Count', 'Total Qty']
    print(side_stats)
    
    # By status
    print("\nTransactions by Status:")
    status_counts = df_transactions['status'].value_counts()
    print(status_counts)
else:
    print("No completed transactions found")

No completed transactions found


## 6. Time-Based Analysis

In [8]:
if len(df_completed) > 0:
    # Convert timestamp to datetime
    df_completed['datetime'] = pd.to_datetime(df_completed['timestamp'])
    df_completed['date'] = df_completed['datetime'].dt.date
    df_completed['hour'] = df_completed['datetime'].dt.hour
    
    print("=" * 60)
    print("TIME-BASED ANALYSIS")
    print("=" * 60)
    
    # Daily transaction count
    print("\nDaily Transaction Count:")
    daily_counts = df_completed.groupby('date').size().tail(10)
    print(daily_counts)
    
    # Hourly distribution
    print("\nHourly Distribution:")
    hourly_dist = df_completed.groupby('hour').size().sort_index()
    print(hourly_dist)
    
    # Trading period
    print(f"\nFirst transaction: {df_completed['datetime'].min()}")
    print(f"Last transaction: {df_completed['datetime'].max()}")
    print(f"Trading period: {(df_completed['datetime'].max() - df_completed['datetime'].min()).days} days")

## 7. P&L Analysis

In [9]:
# Get P&L from database (if tracked)
try:
    if 'conn' in globals() and conn:
        pnl_query = """
        SELECT 
            symbol,
            SUM(CASE WHEN trade_type = 'BUY' THEN -proceeds - commission - fee 
                     ELSE proceeds - commission - fee END) as realized_pnl,
            COUNT(*) as trade_count,
            SUM(quantity) as total_quantity
        FROM trades
        GROUP BY symbol
        """
        
        df_pnl = conn.execute(pnl_query).fetchdf()
        
        if len(df_pnl) > 0:
            print("=" * 60)
            print("PROFIT & LOSS ANALYSIS")
            print("=" * 60)
            
            df_pnl['realized_pnl'] = df_pnl['realized_pnl'].round(2)
            df_pnl = df_pnl.sort_values('realized_pnl', ascending=False)
            
            print("\nRealized P&L by Symbol:")
            print(df_pnl.to_string(index=False))
            
            total_pnl = df_pnl['realized_pnl'].sum()
            print(f"\nTotal Realized P&L: ${total_pnl:,.2f}")
            
            winning_symbols = len(df_pnl[df_pnl['realized_pnl'] > 0])
            losing_symbols = len(df_pnl[df_pnl['realized_pnl'] < 0])
            
            if winning_symbols + losing_symbols > 0:
                win_rate = winning_symbols / (winning_symbols + losing_symbols) * 100
                print(f"Winning Symbols: {win_rate:.1f}% ({winning_symbols}W / {losing_symbols}L)")
        else:
            print("No P&L data available")
    else:
        print("Database connection not available. Please run cell 4 first.")
except Exception as e:
    print(f"P&L analysis not available: {e}")
    import traceback
    traceback.print_exc()

PROFIT & LOSS ANALYSIS

Realized P&L by Symbol:
     symbol  realized_pnl  trade_count  total_quantity
TESTTICKER1        6726.1            2             0.0
TESTTICKER2        3331.0            1            10.0

Total Realized P&L: $10,057.10
Winning Symbols: 100.0% (2W / 0L)


## 8. Recent Activity Monitor

In [ ]:
# Monitor recent activity (last 24 hours)
now = datetime.now(timezone.utc)
yesterday = now - timedelta(days=1)

# Ensure timestamp is datetime and UTC aware
if len(df_transactions) > 0:
    if not pd.api.types.is_datetime64_any_dtype(df_transactions['timestamp']):
        df_transactions['timestamp'] = pd.to_datetime(df_transactions['timestamp'])
    
    if df_transactions['timestamp'].dt.tz is None:
        df_transactions['timestamp'] = df_transactions['timestamp'].dt.tz_localize('UTC')
    else:
        df_transactions['timestamp'] = df_transactions['timestamp'].dt.tz_convert('UTC')

    df_recent = df_transactions[df_transactions['timestamp'] >= yesterday].copy()
else:
    df_recent = pd.DataFrame()

print("=" * 60)
print("RECENT ACTIVITY (Last 24 Hours)")
print("=" * 60)
print(f"\nTotal transactions: {len(df_recent)}")

if len(df_recent) > 0:
    print("\nRecent orders:")
    print(df_recent[['timestamp', 'symbol', 'side', 'price', 'quantity', 'status']].to_string(index=False))
    
    # Volume by symbol
    print("\nVolume by Symbol (Last 24h):")
    recent_volume = df_recent.groupby('symbol')['quantity'].sum().sort_values(ascending=False)
    print(recent_volume)
else:
    print("No recent activity in the last 24 hours")

RECENT ACTIVITY (Last 24 Hours)

Total transactions: 0
No recent activity in the last 24 hours


## 9. Export Data

In [ ]:
# Export transaction data to CSV
export_filename = f"transactions_export_{datetime.now(timezone.utc).strftime('%Y%m%d_%H%M%S')}.csv"
df_transactions.to_csv(export_filename, index=False)
print(f"✓ Transactions exported to: {export_filename}")

# Show file info
print(f"\nExported {len(df_transactions)} transactions")
print(f"File size: {os.path.getsize(export_filename) / 1024:.2f} KB")

✓ Transactions exported to: transactions_export_20251123_134617.csv

Exported 3 transactions
File size: 0.31 KB


In [12]:
# Close database connection
conn.close()
print("✓ Database connection closed")
print("\nAnalysis complete!")

✓ Database connection closed

Analysis complete!
